In [19]:
import cv2
import numpy as np
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential
import pandas as pd

In [20]:
x=[]
y=[]
labels = [0,1,2]
import os
data_path='gestures'
categories=os.listdir(data_path)
code=dict(zip(categories,labels))
for folder in categories:
    folder_path=os.path.join(data_path,folder)
    images=os.listdir(folder_path)
    for imge in images:
        image_path=os.path.join(folder_path,imge)
        imge=cv2.imread(image_path)
        img=cv2.cvtColor(imge,cv2.COLOR_BGR2GRAY)
        resized=cv2.resize(img,(50,50))
        x.append(resized)
        y.append(code[folder])
x=np.array(x)
x=x/255
y=np.array(y)
y = np.reshape(y,(-1,1))
x = np.reshape(x,(-1,2500))
df = pd.DataFrame(x)
df.insert(2500,'Target',y)
df = df.sample(frac = 1)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,Target
613,0.058824,0.996078,1.000000,0.996078,1.000000,1.0,0.996078,1.000000,0.000000,0.003922,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
152,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.011765,...,0.0,0.0,0.0,0.003922,0.007843,0.066667,0.070588,0.070588,0.070588,0
804,0.000000,0.000000,0.003922,0.000000,0.670588,1.0,1.000000,0.996078,0.996078,1.000000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
2622,0.003922,0.000000,0.003922,0.000000,0.007843,0.0,1.000000,0.996078,1.000000,0.992157,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
853,0.000000,0.003922,0.003922,0.003922,0.000000,0.0,0.007843,0.992157,1.000000,0.992157,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0


In [21]:
x = df.iloc[:,0:2500]
x = np.array(x)
y = df['Target']
y = np.array(y)
y = np.reshape(y,(-1,1))
print(np.unique(y))
print(y)

[0 1 2]
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
print(np.unique(y_train))

(3240, 2500) (360, 2500) (3240, 1) (360, 1)
[0 1 2]


In [23]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
x_train = np.reshape(x_train,(-1,50,50))
x_test = np.reshape(x_test,(-1,50,50))

In [24]:
print(y_train.shape,y_train)

(3240, 3) [[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [ ]:
x_train = np.reshape(x_train,(-1,50,50))
x_test = np.reshape(x_test,(-1,50,50))

In [32]:
model = Sequential()
model.add(Conv2D(32, (5,5), input_shape = (50,50,1), activation = 'relu'))
model.add(MaxPooling2D((2,2), (2,2), padding = 'same'))
model.add(Conv2D(64, (5,5), activation = 'sigmoid'))
model.add(MaxPooling2D((5,5), (5,5), padding = 'same'))
model.add(Flatten())
model.add(Dense(1024,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(3,activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 46, 46, 32)        832       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 19, 19, 64)        51264     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)             

In [35]:
model.fit(x=x_train,y=y_train, validation_data = (x_test,y_test), epochs = 5, verbose=2, batch_size = 128)

Train on 3240 samples, validate on 360 samples
Epoch 1/5
 - 7s - loss: 1.4877 - accuracy: 0.4657 - val_loss: 0.4776 - val_accuracy: 0.9972
Epoch 2/5
 - 6s - loss: 0.1179 - accuracy: 0.9994 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 3/5
 - 6s - loss: 7.3448e-04 - accuracy: 1.0000 - val_loss: 1.6749e-04 - val_accuracy: 1.0000
Epoch 4/5
 - 6s - loss: 1.7219e-04 - accuracy: 1.0000 - val_loss: 6.3098e-05 - val_accuracy: 1.0000
Epoch 5/5
 - 6s - loss: 9.2379e-05 - accuracy: 1.0000 - val_loss: 3.3713e-05 - val_accuracy: 1.0000


In [36]:
model.save('handgesmodel.h5')